In [2]:
airline=['virginamerica','united']

In [ ]:
from __future__ import print_function
import io

with io.open('/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/united_negative_151_records.txt','r',encoding='utf-8',errors='ignore') as infile, \
     io.open('/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/united_negative_151_records_decoded.txt','w',encoding='ascii',errors='ignore') as outfile:
    for line in infile:
        print(*line.split(), file=outfile)

In [63]:
data={}
#data['virginamerica'] = open("/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/virg_negative_150_records.txt", "r").read()
#data['united'] = open("/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/united_negative_151_records.txt", "r").read()

# used decoded file to avoid unicode character text
data['virginamerica'] = open("/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/virg_negative_150_records_decoded.txt", "r").read()
data['united'] = open("/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/united_negative_151_records_decoded.txt", "r").read()


In [ ]:
#import csv

#output=open('xyz.txt','w')

#with open('/Users/rajeshgudikoti/Documents/rajesh/learning/data_sets/airline/Tweets_negative.csv',"rt", encoding='ascii') as f:
#for row in f:
    #output.write(row)

In [19]:
#data

In [64]:
next(iter(data.keys()))

'virginamerica'

In [76]:
pd.set_option('max_colwidth',150)
#print(data)
#data_df = pd.DataFrame([data], columns=data.keys()) #***** this also works
data_df = pd.DataFrame.from_dict(data,orient='index')
data_df.columns = ['reviews']
#data_df = data_df.sort_index()
data_df
#print(data_df.reviews.loc['united'])

,reviews
virginamerica,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse\n@VirginAmerica and..."
united,@united #Newarkliberty Airport need to indicate different gate#'s for terminals A&amp;C. #Flyingainteasy\n@united yes please! I am newly married a...


In [77]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation #and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [67]:
data_clean = pd.DataFrame(data_df.reviews.apply(round1))
data_clean

,reviews
virginamerica,virginamerica its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse\nvirginamerica and its a r...
united,united newarkliberty airport need to indicate different gates for terminals aampc flyingainteasy\nunited yes please i am newly married and trying ...


In [68]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [69]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.reviews.apply(round2))
data_clean

,reviews
virginamerica,virginamerica its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recoursevirginamerica and its a rea...
united,united newarkliberty airport need to indicate different gates for terminals aampc flyingainteasyunited yes please i am newly married and trying to...


In [70]:
# Let's pickle it for later use
data_df.to_pickle("pickle/corpus.pkl")

In [71]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.reviews)
#print(data_cv.toarray()[[0]]) ### 7484 columns(number of occurence unique words)
#print(cv.get_feature_names()) ### unique words
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
print(data_clean.index)
data_dtm

Index(['virginamerica', 'united'], dtype='object')


,aampc,ability,able,absolute,access,accompany,according,account,accountable,action,...,youcouldntmakethis,youd,youll,youre,youunited,youve,youvirginamerica,yr,zero,zone
virginamerica,0,1,2,0,1,0,0,2,0,0,...,0,1,0,1,0,0,2,0,0,0
united,1,0,1,1,0,1,1,1,1,1,...,1,0,2,2,3,1,0,1,1,1
